# Spherical harmonics

## References

* [Spherical Harmonics for Beginners](https://dickyjim.wordpress.com/2013/09/04/spherical-harmonics-for-beginners/)
* [Spherical Harmonic Lighting: The Gritty Details](references/spherical-harmonic-lighting.pdf)
* [Spherical Harmonics in Actual Games](https://tomforsyth1000.github.io/papers/papers.html)
* [Stupid Spherical Harmonics (SH) Tricks](http://www.ppsloan.org/publications/StupidSH36.pdf)
* [An Efficient Representation for Irradiance Environment Maps](http://cseweb.ucsd.edu/~ravir/papers/envmap/envmap.pdf)
* [On the relationship between radiance andirradiance:  determiningthe illumination fromimages of a convex Lambertian object](http://cseweb.ucsd.edu/~ravir/papers/invlamb/josa.pdf)
* [GameDev.net - Spherical Harmonics Cubemap](https://www.gamedev.net/forums/topic/671562-spherical-harmonics-cubemap/)
* [HLSL-Spherical-Harmonics](https://github.com/sebh/HLSL-Spherical-Harmonics)  
* [Dive in SH buffer idea](https://seblagarde.wordpress.com/2011/10/09/dive-in-sh-buffer-idea/)

## SH coefficient visualization

In [ ]:
import numpy as np
import sph_harm as sh
import ipyvolume as ipv
from matplotlib import cm, colors


# color map for SH coefficient visualization
N = 256
vals = np.zeros((N, 4))
vals[:N//2, 0] = np.linspace(1, 0, N//2)
vals[N//2:, 1] = np.linspace(0, 1, N//2)
vals[:, 3] = np.ones(N)
sh_colormap = colors.ListedColormap(vals)

theta = np.linspace(0, np.pi, num=128)
phi = np.linspace(0, 2*np.pi, num=128)
theta, phi = np.meshgrid(theta, phi)

x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)

sh_band = 5

ipv.figure(lighting=False)
ipv.xyzlim(-sh_band*2, sh_band*2)
ipv.ylim(-sh_band*3, sh_band)

for l in np.arange(sh_band):
    for m in np.arange(-l, l+1):
        sh_vals = sh.sph_harm(m, l, theta, phi).real
        abs_sh = np.abs(sh_vals)
        sh_min, sh_max = sh_vals.min(), sh_vals.max()
        sh_max += 0.000001
        sh_gradient = (sh_vals - sh_min) / (sh_max - sh_min)

        cmap = sh_colormap(sh_gradient)
        if l == 0:
            # hack the cmap for first order cause it's a constant so that gradient is zero
            cmap = "green"

        coord_offset = np.array([2*m, -2*l, 0])
        ipv.plot_mesh(x*abs_sh+coord_offset[0], z*abs_sh+coord_offset[1], y*abs_sh+coord_offset[2], wireframe=False, color=cmap)
        sphere_scale = 0.2
        sphere_offset = coord_offset+np.array([0.5, 0.5, 0])
        ipv.plot_mesh(x*sphere_scale+sphere_offset[0], z*sphere_scale+sphere_offset[1], y*sphere_scale+sphere_offset[2], wireframe=False, color=cmap)
ipv.show()

## SH projection example

Define a function that we want to project into SH basis

In [ ]:
import numpy as np
import sph_harm as sh
import ipyvolume as ipv
from matplotlib import cm

def func(theta, phi):
    l = np.maximum(0, 5*np.cos(theta)-4) + np.maximum(0, -4*np.sin(theta-np.pi)*np.cos(phi-2.5+np.pi)-3)
    return l

Visualize it

In [ ]:
theta = np.linspace(0, np.pi, num=64)
phi = np.linspace(0, 2*np.pi, num=128)
theta, phi = np.meshgrid(theta, phi)

x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)

vals = func(theta, phi)

ipv.figure(lighting=False)
ipv.xyzlim(-1, 1)
ipv.plot_mesh(x, z, y, wireframe=False, color=cm.coolwarm(vals))
ipv.show()

Another way to visualize it

In [ ]:
ipv.figure(lighting=False)
ipv.xyzlim(-1, 1)
ipv.plot_mesh(x*vals, z*vals, y*vals, wireframe=False, color=cm.coolwarm(vals))
ipv.show()

Now define SH projection and reconstruction functions

In [ ]:
def sh_projection(spherical_func, band, numsamples):
    sh_coeffs = np.ndarray(band**2)
    sh_coeffs.fill(0)

    for l in range(band):
        for m in range(-l, l + 1):
            dtheta = np.pi / (numsamples - 1)
            dphi = 2 * np.pi / (numsamples - 1)
            sh_coeff = 0
            for i in np.arange(0, numsamples):
                for j in np.arange(0, numsamples):
                    theta = dtheta * i
                    phi = dphi * j
                    v = spherical_func(theta, phi)
                    sh_val = sh.sph_harm(m, l, theta, phi).real
                    sh_coeff += sh_val * v * np.sin(theta) * dtheta * dphi

            i = l * (l + 1) + m
            sh_coeffs[i] = sh_coeff
    return sh_coeffs

def sh_reconstruction(theta, phi, band, sh_coeffs):
    vals = np.zeros_like(theta)
    for l in range(band):
        for m in range(-l, l + 1):
            sh_val = sh.sph_harm(m, l, theta, phi).real
            i = l * (l + 1) + m
            coeff = sh_coeffs[i]
            vals = vals + sh_val * coeff
    return vals

Visualize the difference between original function and reconstructed function

In [ ]:
theta = np.linspace(0, np.pi, num=64)
phi = np.linspace(0, 2*np.pi, num=128)
theta, phi = np.meshgrid(theta, phi)

x = np.sin(theta) * np.cos(phi)
y = np.sin(theta) * np.sin(phi)
z = np.cos(theta)

sh_band = 6
coeff = sh_projection(func, sh_band, 128)
diff = sh_reconstruction(theta, phi, sh_band, coeff) - func(theta, phi)

colormap = cm.coolwarm 
color = colormap(diff)

ipv.figure(lighting=False)
ipv.xyzlim(-1, 1)
ipv.plot_mesh(x, z, y, wireframe=False, color=color)
ipv.show()